# Preprocessing of Imaging data
<i>**Latest update</i> - Dec 2024**  

#### **Authors:**  
[Oscar Dong](https://github.com/Awesomenous) (oscardong4@gmail.com) | [Thomas O'Neil](https://github.com/DrThomasOneil) (thomas.oneil@sydney.edu.au) | [Heeva Baharlou](heeva.baharlou@sydney.edu.com)  

##### The purpose of this notebook is to provide a consolidated approach to IMC analysis and forms the prerequisite steps to the IMComplete R package workflow. We focused 
